In [105]:
import sys
from datetime import date
from typing import List

import chromedriver_autoinstaller
import pandas as pd
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver

In [104]:

class ExtractMissingPeople:

    LOADED_PAGES: int = 100
    URL: str = f"https://www.missingpeople.org.uk/appeal-search?loaded={LOADED_PAGES}"

    def __init__driver(self) -> webdriver.Chrome:

        chromedriver_autoinstaller.install()
        self.driver = webdriver.Chrome()

    def accept_cookies_banner(self):

        cookie_banner = WebDriverWait(
            self.driver, 5).until(
                ec.visibility_of_any_elements_located(
                    (By.ID, 'cookie-law-info-bar')))[0]

        accept_button = WebDriverWait(
            cookie_banner, 5).until(
                ec.visibility_of_any_elements_located(
                    (By.TAG_NAME, 'a')))[-1]

        accept_button.click()

    def close_pop_up(self):

        pop_up = WebDriverWait(
            self.driver, 5).until(
                ec.visibility_of_any_elements_located(
                    (By.CLASS_NAME, 'close')))[0]

        pop_up.click()
    
    def extract_all_urls(self) -> List[str]:

        self.driver.get(url=self.URL)

        self.url_list = []
        all_content = WebDriverWait(
            self.driver, 5).until(
                ec.visibility_of_any_elements_located(
                    (By.CLASS_NAME, 'section__content')))

        all_a_tags = WebDriverWait(
            all_content[0], 5).until(
                ec.visibility_of_any_elements_located(
                    (By.TAG_NAME, 'a')))

        for url in all_a_tags:
            self.url_list.append(url.get_attribute("href"))

    def extract_all_data_from_urls(self) -> pd.DataFrame:
    
        df_full = pd.DataFrame()
        for url in self.url_list[:4]:

            self.driver.get(url=url)
            self.close_pop_up(driver=self.driver)

            self.item_dict = {}

            # Extract the main content
            main_content = WebDriverWait(
                self.driver, 5).until(
                    ec.visibility_of_any_elements_located(
                        (By.CLASS_NAME, 'main_content_cell')))[0]

            # Get name
            self.item_dict["NAME"] = WebDriverWait(
                main_content, 5).until(
                    ec.visibility_of_any_elements_located(
                        (By.TAG_NAME, 'h1')))[0].text

            item_content = WebDriverWait(
                main_content, 5).until(
                    ec.visibility_of_any_elements_located(
                        (By.TAG_NAME, 'li')))

            for item in item_content[:-2]:

                item_name = WebDriverWait(
                    item, 5).until(
                        ec.visibility_of_any_elements_located(
                            (By.TAG_NAME, 'h2')))[0].text
                item_value = WebDriverWait(
                        item, 5).until(
                            ec.visibility_of_any_elements_located(
                                (By.TAG_NAME, 'span')))[0].text

                self.item_dict[item_name] = item_value

        df_full = pd.concat([df_full, pd.DataFrame(self.item_dict, index=[0])])

    def run(self):

        # Initalise driver
        self.__init__driver()

        # Extract all URLs
        self.extract_all_urls()

        # Extract data frame
        self.extract_all_data_from_urls()


In [98]:
df_full = pd.DataFrame()
for url in url_list[:4]:

    driver.get(url=url)
    close_pop_up(driver=driver)

    item_dict = {}

    # Extract the main content
    main_content = WebDriverWait(
        driver, 5).until(
            ec.visibility_of_any_elements_located(
                (By.CLASS_NAME, 'main_content_cell')))[0]

    # Get name
    item_dict["NAME"] = WebDriverWait(
        main_content, 5).until(
            ec.visibility_of_any_elements_located(
                (By.TAG_NAME, 'h1')))[0].text

    item_content = WebDriverWait(
        main_content, 5).until(
            ec.visibility_of_any_elements_located(
                (By.TAG_NAME, 'li')))

    for item in item_content[:-2]:

        item_name = WebDriverWait(
            item, 5).until(
                ec.visibility_of_any_elements_located(
                    (By.TAG_NAME, 'h2')))[0].text
        item_value = WebDriverWait(
                item, 5).until(
                    ec.visibility_of_any_elements_located(
                        (By.TAG_NAME, 'span')))[0].text

        item_dict[item_name] = item_value

    df_full = pd.concat([df_full, pd.DataFrame(item_dict, index=[0])])

In [99]:
df_full

,NAME,AGE AT DISAPPEARANCE,MISSING FROM,MISSING SINCE,REFERENCE NO
0,Russell Hardy,57,"Stratford Upon Avon, Warwickshire",27/06/2023,23-002791
0,Jamie Reid,44,"Hamilton, Lanarkshire",24/06/2023,23-002726
0,Christopher Grass,32,"Poole, Dorset",12/06/2023,23-002713
0,Fahad Rashid,43,"South Bank, Middlesbrough",20/05/2023,23-002712


In [67]:
item_dict

{'NAME': 'Russell Hardy',
 'AGE_AT_DISAPPEARANCE': '57',
 'MISSING_FROM': 'Stratford Upon Avon, Warwickshire',
 'MISSING_SINCE': '27/06/2023',
 'REFERENCE_NO': '23-002791'}

In [75]:
df

,NAME,AGE_AT_DISAPPEARANCE,MISSING_FROM,MISSING_SINCE,REFERENCE_NO
0,Russell Hardy,57,"Stratford Upon Avon, Warwickshire",27/06/2023,23-002791
